In [1]:
import pandas as pd

In [161]:
data = pd.read_csv("../datasets/ml-100k/u.data.csv", sep="\t", header=None)
data.columns = ["UserID", "MovieID", "Rating", "TimeStamp"]
data.drop(["TimeStamp"], axis=1, inplace=True)
data.head()    

,UserID,MovieID,Rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [162]:
movies = pd.read_csv("../datasets/ml-100k/u.item.csv", sep="|", encoding="ISO-8859-1", header=None)
movies = movies.iloc[:,:2]
movies.columns = ["MovieID", "Title"]
movies.head()

,MovieID,Title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [163]:
print(data.shape)
print(movies.shape)

(100000, 3)
(1682, 2)


In [196]:
#Juntamos las dos tablas
ratings = pd.merge(movies, data)
print(ratings.shape)
ratings.head()

(100000, 4)


,MovieID,Title,UserID,Rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [164]:
#Pivotamos la tabla para crear una nueva tabla con una fila para cada usuario y una celda para cada película
movie_ratings = ratings.pivot_table(index="UserID", columns=["Title", values="Rating")
movie_ratings.head(10)

Title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
UserID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN
6,NaN,NaN,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,4.0,NaN,NaN,5.0,5.0,NaN,4.0,...,NaN,NaN,NaN,5.0,3.0,NaN,3.0,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Filtrado colaborativo por ítems
* Si quisiéramos hacer filtrado colaborativo por usuarios, tendríamos que coger al usuario (una fila)
* Como queremos hacer filtrado colaborativo por ítems (películas), hay que coger columnas y correlacionarlas entre sí

In [165]:
#Vamos a buscar películas parecidas a Toy Story
toyStory_ratings = movie_ratings["Toy Story (1995)"]
toyStory_ratings.head()

UserID
1    5.0
2    4.0
3    NaN
4    NaN
5    4.0
Name: Toy Story (1995), dtype: float64

In [166]:
#Correlacionamos el resto de películas (columnas) con Toy Story
similar_movies = movie_ratings.corrwith(toyStory_ratings)
similar_movies.dropna(inplace=True)
similar_movies.head()

/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2392: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2326: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


Title
'Til There Was You (1997)      0.534522
101 Dalmatians (1996)          0.232118
12 Angry Men (1957)            0.334943
187 (1997)                     0.651857
2 Days in the Valley (1996)    0.162728
dtype: float64

In [173]:
df = pd.DataFrame(similar_movies)
df.columns = ["Correlation"]
df.head(10)

,Correlation
Title,
'Til There Was You (1997),0.534522
101 Dalmatians (1996),0.232118
12 Angry Men (1957),0.334943
187 (1997),0.651857
2 Days in the Valley (1996),0.162728
"20,000 Leagues Under the Sea (1954)",0.328472
2001: A Space Odyssey (1968),-0.069060
"39 Steps, The (1935)",0.150055
8 1/2 (1963),-0.117259


In [75]:
import numpy as np

In [168]:
movieStats = ratings.groupby("Title").count()
movieStats.head(10)

,MovieID,UserID,Rating
Title,,,
'Til There Was You (1997),9,9,9
1-900 (1994),5,5,5
101 Dalmatians (1996),109,109,109
12 Angry Men (1957),125,125,125
187 (1997),41,41,41
2 Days in the Valley (1996),93,93,93
"20,000 Leagues Under the Sea (1954)",72,72,72
2001: A Space Odyssey (1968),259,259,259
3 Ninjas: High Noon At Mega Mountain (1998),5,5,5


In [169]:
#Nos quedamos con las películas con más de 100 valoraciones
popularMovies = movieStats[movieStats.Rating >= 100]
popularMovies.drop(["MovieID", "UserID"], axis=1, inplace=True)
popularMovies.columns = ["n_ratings"]
popularMovies.head(10)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,n_ratings
Title,
101 Dalmatians (1996),109
12 Angry Men (1957),125
2001: A Space Odyssey (1968),259
Absolute Power (1997),127
"Abyss, The (1989)",151
Ace Ventura: Pet Detective (1994),103
"Adventures of Priscilla, Queen of the Desert, The (1994)",111
"African Queen, The (1951)",152
Air Force One (1997),431


In [156]:
popularMovies.shape

(338, 2)

In [157]:
df.shape

(1370, 1)

In [187]:
final_data = pd.concat([df, popularMovies], axis=1)
final_data = final_data[final_data.n_ratings >= 100]
final_data.sort_values(["Correlation"], ascending=False, inplace=True)
final_data.head(10)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Correlation,n_ratings
Toy Story (1995),1.000000,452.0
"Craft, The (1996)",0.549100,104.0
Down Periscope (1996),0.457995,101.0
Miracle on 34th Street (1994),0.456291,101.0
G.I. Jane (1997),0.454756,175.0
Amistad (1997),0.449915,124.0
Beauty and the Beast (1991),0.442960,202.0
"Mask, The (1994)",0.432855,129.0
Cinderella (1950),0.428372,129.0
That Thing You Do! (1996),0.427936,176.0


## Construir el modelo de recomendación
* Ahora que hemos visto como encontrar similitudes entre películas, vamos a generar recomendaciones para un usuario, basadas en su actividad anterior (histórico de puntuaciones).
* En vez de correlacionar la película Toy Story con las demás, vamos a correlacionar todas con todas.

In [191]:
#Usamos el método de Pearson
corrMatrix = movie_ratings.corr(method="pearson", min_periods=100)
corrMatrix.head()

Title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
Title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-900 (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1996),NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 Angry Men (1957),NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187 (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Cada celda representa lo que una película se parece a otra

In [237]:
#Cogemos un usuario cualquiera para generar recomendaciones según lo que ha visto
#Eliminamos los na porque son las películas que no ha visto

usuario = movie_ratings.loc[105].dropna()
usuario

Title
Alien: Resurrection (1997)         2.0
Boogie Nights (1997)               3.0
Chasing Amy (1997)                 4.0
Contact (1997)                     5.0
Cop Land (1997)                    4.0
Devil's Advocate, The (1997)       2.0
English Patient, The (1996)        4.0
Full Monty, The (1997)             4.0
Game, The (1997)                   3.0
Gattaca (1997)                     5.0
Good Will Hunting (1997)           4.0
L.A. Confidential (1997)           5.0
Lost Highway (1997)                4.0
Mimic (1997)                       2.0
Replacement Killers, The (1998)    3.0
Saint, The (1997)                  2.0
Scream (1996)                      4.0
Seven Years in Tibet (1997)        3.0
Soul Food (1997)                   3.0
Starship Troopers (1997)           2.0
Titanic (1997)                     5.0
Tomorrow Never Dies (1997)         2.0
Wag the Dog (1997)                 3.0
Name: 105, dtype: float64

In [239]:
corrMatrix[usuario.index[4]].dropna()

Title
Air Force One (1997)           0.210321
Contact (1997)                 0.069685
Cop Land (1997)                1.000000
English Patient, The (1996)    0.066189
L.A. Confidential (1997)       0.244052
Liar Liar (1997)               0.266906
Scream (1996)                  0.211042
Name: Cop Land (1997), dtype: float64

* Para cada película que el usuario ha puntuado, cogemos todas las películas similares según nuestra matriz
de similaridad anterior.
* Para evitar que le recomendemos simplemente peliculas parecidas, y le mostremos 
peliculas parecidas QUE LE GUSTARÁN, lo que haremos será potenciar el valor de correlación entre películas 
en función de la valoración que el usuario asignó a las que vió, multiplicando la puntuación del usuario por la 
correlación del par de películas. 
* De este modo, además, las peliculas que no le gustaron (como Tomorrow Never Dies) hará que las peliculas parecidas tengan peor puntuación.

In [243]:
#NO FUNCIONA BIEN

posiblesSimilares = pd.Series()

#Recorremos las películas vistas por el usuario
for i in range(0, len(usuario.index)):
    #Obtenemos películas similares a esta que el usuario a puntuado
    similares = corrMatrix[usuario.index[i]].dropna()
    
    #Multiplicamos el score de correlación por la puntuación asignada por el usuario
    similares = similares.map(lambda x: x*usuario[i])
    
    #Añadimos la película y la nueva puntuación a nuestra lista de candidatos
    posiblesSimilares = posiblesSimilares.append(similares)
    
    # Agrupamos los resultados, ya que si una película es parecida a dos de las
    # que ha visto el usuario, aparecerá dos veces
    # podríamos agrupar haciendo la media de puntuación, o como en este caso
    # sumándolo, ya que si aparece dos veces será porque es una película
    # muy recomendable... y así saldrá la primera. Va en cuestión de gustos.
    posiblesSimilares = posiblesSimilares.groupby(posiblesSimilares.index).sum()
    
    # Finalmente filtramos todas aquellas películas que el usuario ya había valorado
    # puesto que no tiene sentido que se las recomendemos si ya las ha visto
    # le decimos que ignore errores para evitar excepciones si hay problemas 
    # con el CharSet de los títulos
    filtered = posiblesSimilares.drop(usuario.index, errors="ignore")
    print(filtered)

Series([], dtype: float64)
Air Force One (1997)   -0.426503
dtype: float64
Air Force One (1997)               -0.442573
Conspiracy Theory (1997)            0.027968
Empire Strikes Back, The (1980)     1.307150
Fargo (1996)                        0.931916
Godfather, The (1972)               1.039976
In & Out (1997)                     1.353538
Independence Day (ID4) (1996)       0.000399
Jerry Maguire (1996)                0.492343
Leaving Las Vegas (1995)            0.826997
Liar Liar (1997)                    0.548275
Mission: Impossible (1996)         -0.432649
Princess Bride, The (1987)          0.082707
Pulp Fiction (1994)                 1.065693
Raiders of the Lost Ark (1981)      0.404005
Return of the Jedi (1983)           1.005698
Rock, The (1996)                    1.173614
Silence of the Lambs, The (1991)    0.194723
Star Trek: First Contact (1996)     0.157428
Star Wars (1977)                    0.298144
Toy Story (1995)                    0.844475
Trainspotting (1996)     

In [241]:
filtered.shape

(169,)